In [1]:
#!unzip /datasets/dfu/dfuSegmentation.zip

In [1]:
!pip install albumentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 MB 32.3 MB/s eta 0:00:0000:0100:01


In [2]:
import os

def ReadFiles(directory):
    for root, dirs, files in os.walk(directory):
        files_ = sorted([f'{directory}/{file}' for file in files], key=str)
        return files_

In [3]:
import cv2
from torch.utils.data import Dataset, DataLoader
import torch
import albumentations as A

class DFUDataset(Dataset):
    """ Custom dataset initiation for diabetic foot ulcer segmentation dataset. """ 

    def __init__(self, image_dir, mask_dir, transform=None):
        self.transform = transform
        self.images, self.masks = image_dir, mask_dir

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image, mask = cv2.imread(self.images[idx]), cv2.imread(self.masks[idx], cv2.IMREAD_GRAYSCALE)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            transformed = self.transform(image=image, mask=mask)
            image, mask = transformed['image'].to(torch.float32) / 255, transformed['mask'].to(torch.float32) / 255

        return image, mask

In [4]:
import torchvision
import torch
import torch.nn.functional as F
import numpy as np
from torch.optim.lr_scheduler import CosineAnnealingLR

#PyTorch
class DiceLoss(torch.nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1e-5):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                         
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice
    
def BCEWithLogitsLoss():
    return torch.nn.BCEWithLogitsLoss()


def IOU(outputs, labels, device):
    output = torch.sigmoid(outputs)
    output = torch.where(output < 0.5, torch.tensor(0.0).to(device), torch.tensor(1.0).to(device)).flatten()
    label = labels.flatten()
    
    intersection = torch.logical_and(label, output)
    union = torch.logical_or(label, output)
    
    return torch.sum(intersection) / torch.sum(union)

def SGD(model, lr=0.1):
    return torch.optim.SGD(params=model.parameters(), lr=lr)

def Adam(model, lr=0.1):
    return torch.optim.Adam(params = model.parameters(), lr=lr)



In [5]:
from tqdm import tqdm
from datetime import datetime
import torch
import csv
import numpy as np

from tqdm import tqdm
from datetime import datetime
import torch
import time
import csv
import numpy as np

def Train(model, epochs, optimizer, scheduler, loss_fn, train_iter, val_iter, device, early_stopping, DeepLab=False):
    # Create dictionary to store history
    Loss = {"val_iou":[]}

    with open('./models/readme.txt', 'w') as f:
        f.write('Model: {} \n Max Epochs: {} \n Optimizer: {} \n Scheduler: {} \n Loss Function: {} \n Device: {} \n Early Stopping: {}'.format(model, epochs, optimizer, scheduler, loss_fn, device, early_stopping))

    with open('./models/log.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["epoch","train_loss","train_iou","val_loss","val_iou"])
    
    # Set patience to zero.
    patience = 0

    for epoch in range(epochs):
        # Set model in training mode
        model.train()

        # Initialise cumulative loss
        train_loss, train_iou, val_loss, val_iou = 0, 0, 0, 0
        
        # Print LR if it has decreased.
        if epoch != 0:
            if optimizer.param_groups[0]['lr'] < LR:
                print('Learning rate decreased to ', optimizer.param_groups[0]['lr'])
        else:
            print('Initial learning rate set to ', optimizer.param_groups[0]['lr'])
        LR = optimizer.param_groups[0]['lr']

        # Loop over the training set
        for i, data in enumerate(tqdm(train_iter)):
            inputs, labels = data[0].to(device), data[1].to(device)

            # Zero previous gradients
            optimizer.zero_grad()
            
            
            # Generate predictions and loss with current model parameters
            if DeepLab == True:
                outputs = model(inputs)["out"]
            else:
                outputs = model(inputs)
            loss = loss_fn(outputs, labels)

            # Initiate backpropagation to adjust loss weights
            loss.backward()
            optimizer.step()

            # Update total training loss
            train_loss += loss
            train_iou += IOU(outputs, labels, device)
        train_steps = i+1

        with torch.no_grad():
            # Set the model to evaluation mode
            model.eval()

            # Loop over the validation set
            for i, data in enumerate(tqdm(val_iter)):
                inputs, labels = data[0].to(device), data[1].to(device)

                # Calculate validation loss
                if DeepLab == True:
                    outputs = model(inputs)["out"]
                else:
                    outputs = model(inputs)
                val_loss += loss_fn(outputs, labels)
                val_iou += IOU(outputs, labels, device)
            val_steps = i+1
        
        # Calculate the average training and validation loss
        avg_train_loss = float(train_loss / train_steps)
        avg_train_iou = float(train_iou / train_steps)
        avg_val_loss = float(val_loss / val_steps)
        avg_val_iou = float(val_iou / val_steps)
        
        if scheduler is not None:
            scheduler.step(avg_val_loss)

        # Save the best model if appropriate, else continue.
        if epoch == 0:
            torch.save(model.state_dict(), "./models/model.pth")
            print("Saved best model!")
        elif avg_val_iou > np.max(Loss["val_iou"]):
            torch.save(model.state_dict(), './models/model.pth')
            print("Saved best model!")
            patience = 0
        else:
            patience += 1

        # Update train and val loss history
        Loss["val_iou"].append(avg_val_iou)

        with open('./models/log.csv', 'a') as csv_file:
            dict_object = csv.DictWriter(csv_file, fieldnames=["epoch","train_loss","train_iou","val_loss","val_iou"])
            dict_object.writerow({"epoch":epoch,"train_loss":avg_train_loss,"train_iou":avg_train_iou,"val_loss":avg_val_loss,"val_iou":avg_val_iou})

        print("Epoch {}, Train Loss {:3f}, Train IOU {:3f}, Val Loss {:3f}, Val IOU {:3f}".format(
            epoch, avg_train_loss, avg_train_iou, avg_val_loss, avg_val_iou))
        
        if patience > early_stopping:
            print("Early stopping triggered, best val IOU: {}".format(np.max(Loss["val_iou"])))
            break

In [6]:
import torchvision
import torch
import torch.nn.functional as F
import numpy as np


class Conv2dBlock(torch.nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(in_c, out_c, kernel_size=3, padding='same')
        self.bn1 = torch.nn.BatchNorm2d(out_c)
        self.conv2 = torch.nn.Conv2d(out_c, out_c, kernel_size=3, padding='same')
        self.bn2 = torch.nn.BatchNorm2d(out_c)
        self.relu = torch.nn.ReLU()
    
    def forward(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x

class Conv2dTransposeBlock(torch.nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.conv1 = torch.nn.ConvTranspose2d(in_channels=in_c, out_channels=out_c, kernel_size=(3,3))
        self.bn1 = torch.nn.BatchNorm2d(out_c)
        self.conv2 = torch.nn.Conv2d(out_c, out_c, kernel_size=3, padding=1)
        self.bn2 = torch.nn.BatchNorm2d(out_c)
        self.relu = torch.nn.ReLU()
    
    def forward(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x

#conv = torch.nn.Conv2d(in_channels= 3, out_channels= 16, kernel_size=(3,3), padding='same')

class UNet(torch.nn.Module):

    def __init__(self):
        super(UNet, self).__init__()
        channels = [3, 16, 32, 64, 128, 256]

        # Conolution layers where conv2d_1 = First Layer
        self.conv2d_1 = Conv2dBlock(in_c=channels[0],out_c=channels[1])
        self.conv2d_2 = Conv2dBlock(in_c=channels[1],out_c=channels[2])
        self.conv2d_3 = Conv2dBlock(in_c=channels[2],out_c=channels[3])
        self.conv2d_4 = Conv2dBlock(in_c=channels[3],out_c=channels[4])
        self.conv2d_5 = Conv2dBlock(in_c=channels[4],out_c=channels[5])

        # Decoding layers for upsampling where conv2dTranspose_1 = First Layer
        self.conv2dTranspose_1 = torch.nn.ConvTranspose2d(in_channels=channels[5], out_channels=channels[4], kernel_size=3, stride=2, padding=1)
        self.conv2d_6 = Conv2dBlock(in_c=channels[5], out_c=channels[4])
        self.conv2dTranspose_2 = torch.nn.ConvTranspose2d(in_channels=channels[4], out_channels=channels[3], kernel_size=3, stride=2, padding=1)
        self.conv2d_7 = Conv2dBlock(in_c=channels[4], out_c=channels[3])
        self.conv2dTranspose_3 = torch.nn.ConvTranspose2d(in_channels=channels[3], out_channels=channels[2], kernel_size=3, stride=2, padding=1)
        self.conv2d_8 = Conv2dBlock(in_c=channels[3], out_c=channels[2])
        self.conv2dTranspose_4 = torch.nn.ConvTranspose2d(in_channels=channels[2], out_channels=channels[1], kernel_size=3, stride=2, padding=1)
        self.conv2d_9 = Conv2dBlock(in_c=channels[2], out_c=channels[1])

        # Define max pooling and dropout functions
        self.maxPool = torch.nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.dropout = torch.nn.Dropout(p=0.1, inplace=False)
        
        # Classify prediction mask to single channel
        self.segment = torch.nn.Conv2d(channels[1], 1, kernel_size=1, padding=0)
        self.activation = torch.nn.Sigmoid()

    def forward(self, x):
        # Encoding block
        enc = []
        for conv in [self.conv2d_1, self.conv2d_2, self.conv2d_3, self.conv2d_4]:
            x = conv(x)
            enc.append(x)
            x = self.maxPool(x)
            #x = self.dropout(x)
        x = self.conv2d_5(x)

        # Decoding block
        for i, l in enumerate([[self.conv2dTranspose_1, self.conv2d_6], [self.conv2dTranspose_2, self.conv2d_7], 
                                [self.conv2dTranspose_3,self.conv2d_8], [self.conv2dTranspose_4, self.conv2d_9]]):
            trans, conv = l[0], l[1]
            x = trans(x, output_size=((x.size()[2])*(2), x.size()[3]*(2)))
            x = torch.cat((x, enc[3-i]), axis=1)
            #x = self.dropout(x)
            x = conv(x)
        
        x = self.segment(x)
        x = torch.squeeze(x)
        #x = self.activation(x)

        return x


class UNet2(torch.nn.Module):

    def __init__(self):
        super(UNet2, self).__init__()
        channels = [3, 16, 32, 64, 128, 256]

        # Conolution layers where conv2d_1 = First Layer
        self.conv2d_1 = Conv2dBlock(in_c=channels[0],out_c=channels[1])
        self.conv2d_2 = Conv2dBlock(in_c=channels[1],out_c=channels[2])
        self.conv2d_3 = Conv2dBlock(in_c=channels[2],out_c=channels[3])
        self.conv2d_4 = Conv2dBlock(in_c=channels[3],out_c=channels[4])
        self.conv2d_5 = Conv2dBlock(in_c=channels[4],out_c=channels[5])

        # Decoding layers for upsampling where conv2dTranspose_1 = First Layer
        self.conv2dTranspose_1 = torch.nn.ConvTranspose2d(in_channels=channels[5], out_channels=channels[4], kernel_size=3, stride=2, padding=1)
        self.conv2d_6 = Conv2dBlock(in_c=channels[5], out_c=channels[4])
        self.conv2dTranspose_2 = torch.nn.ConvTranspose2d(in_channels=channels[4], out_channels=channels[3], kernel_size=3, stride=2, padding=1)
        self.conv2d_7 = Conv2dBlock(in_c=channels[4], out_c=channels[3])
        self.conv2dTranspose_3 = torch.nn.ConvTranspose2d(in_channels=channels[3], out_channels=channels[2], kernel_size=3, stride=2, padding=1)
        self.conv2d_8 = Conv2dBlock(in_c=channels[3], out_c=channels[2])
        self.conv2dTranspose_4 = torch.nn.ConvTranspose2d(in_channels=channels[2], out_channels=channels[1], kernel_size=3, stride=2, padding=1)
        self.conv2d_9 = Conv2dBlock(in_c=channels[2], out_c=channels[1])

        # Define max pooling and dropout functions
        self.maxPool = torch.nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.dropout = torch.nn.Dropout(p=0.1, inplace=False)
        
        # Classify prediction mask to single channel
        self.segment = torch.nn.Conv2d(channels[1], 1, kernel_size=1, padding=0)

    def forward(self, x):
        # Encoding block
        enc = []
        for conv in [self.conv2d_1, self.conv2d_2, self.conv2d_3]:
            x = conv(x)
            enc.append(x)
            x = self.maxPool(x)
            x = self.dropout(x)
        x = self.conv2d_4(x)

        # Decoding block
        for i, l in enumerate([[self.conv2dTranspose_2, self.conv2d_7], 
                                [self.conv2dTranspose_3,self.conv2d_8], [self.conv2dTranspose_4, self.conv2d_9]]):
            trans, conv = l[0], l[1]
            x = trans(x, output_size=((x.size()[2])*(2), x.size()[3]*(2)))
            x = torch.cat((x, enc[2-i]), axis=1)
            x = self.dropout(x)
            x = conv(x)
        
        x = self.segment(x)
        x = torch.squeeze(x)

        return x

def DeepLabModel():
    model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained=True)
    model.classifier[4] = torch.nn.Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1)) # Change final layer to 1 class.
    return model

def DeepLabV3_MobileNet_V3_Large():
    model = torchvision.models.segmentation.deeplabv3_mobilenet_v3_large(weights=None, num_classes=1)
    return model



In [ ]:
import collections
import math
import os
import shutil
import pandas as pd
import torch
import torchvision
from torch import nn
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import random
import cv2
from tqdm import tqdm
#import IProgress
from datetime import datetime

from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor, Resize
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

def Configurations():
    """ 
    Set the batch size for the data loader, large batch sizes can run into problems with memory.
    Suggested batch sizes: 2, 4, 8, 16
    """
    batch_size = 16

    """
    Set the maximum number of epochs for the training phase, default setting of 1000. Model is likely
    to converge before 1000 epochs when early stopping is set.
    """
    max_epochs = 1000

    """
    Set the patience for early stopping, defualt 15. The training phase will end once the validation IOU
    has not increased for the number of epochs set in early stopping.
    """
    early_stopping = 15

    """
    Set the model architecture for deep learning.
    Options: 
    'DeepLabModel()'
    'UNet()'
    'DeepLabV3_MobileNet_V3_Large()'
    """
    #model = UNet()
    model = DeepLabV3_MobileNet_V3_Large()
    
    DeepLab = True

    """
    Set the model optimizer for training the selected model.
    Options:
    'SGD(model)'
    'Adam(model)'
    """
    if batch_size == 2:
        optimizer = Adam(model, lr=0.0001)
    elif batch_size == 16:
        optimizer = SGD(model, lr=0.01)
    else:
        optimizer = Adam(model, lr=0.0001)

    """
    Set the learning rate scheduler for the optimization function. The default selection is
    'ReduceLROnPlateau' which decreases the learning rate by a factor of 10 when val IOU has not
    increased for a patience of 5 epochs
    """
    #scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5,
    #                                                       patience=5, cooldown=5, min_lr=1e-6)
    scheduler = None

    """
    Set the loss function for training the model, default Dice.
    Options:
    'DiceLoss()'
    'BCEWithLogitsLoss()'
    """
    loss_fn = DiceLoss()

    return batch_size, max_epochs, early_stopping, model, optimizer, scheduler, loss_fn, DeepLab

def ReadDirectories():
    print("Reading files from directories.")
    data_path = "/notebooks/dfuSegmentation"
    # Set training directory
    train_dir = os.path.join(data_path, "dfuc2022/train/images")
    train_mask_dir = os.path.join(data_path, "dfuc2022//train/masks")

    # Set validation directory
    val_dir = os.path.join(data_path, "dfuc2022/val/images")
    val_mask_dir = os.path.join(data_path, "dfuc2022/val/masks")

    # Set test directory
    test_dir = os.path.join(data_path, "dfuc2022/test/")

    train_files = ReadFiles(train_dir)
    train_masks = ReadFiles(train_mask_dir)

    val_files = ReadFiles(val_dir)
    val_masks = ReadFiles(val_mask_dir)

    test_files = ReadFiles(test_dir)
    print("Complete.")

    return train_files, train_masks, val_files, val_masks, test_files

def Transforms():
    transform_train = A.Compose([
        #A.augmentations.dropout.cutout.Cutout(num_holes=8, max_h_size=8, max_w_size=8, fill_value=0, always_apply=False, p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        #A.RandomBrightnessContrast(p=0.2),
        #A.augmentations.transforms.GaussNoise(var_limit=(100.0, 5000.0), mean=0, per_channel=True, always_apply=False, p=0.5),
        A.augmentations.geometric.transforms.ShiftScaleRotate(
            shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, interpolation=1, p=0.5),
        A.augmentations.geometric.rotate.Rotate(limit=90, p=0.5),
        A.augmentations.crops.transforms.CropAndPad(percent=(-0.3, 0.05), keep_size=True, p=0.5),
        #A.augmentations.dropout.coarse_dropout.CoarseDropout(max_holes=50, max_height=8, max_width=8, p=0.5),
        #A.augmentations.geometric.transforms.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, interpolation=1, border_mode=4, p=0.5),
        ToTensorV2(),
    ])

    transform_test = A.Compose([
        ToTensorV2(),
    ])

    return transform_train, transform_test

def main():
    # Read directories into file lists.
    train_files, train_masks, val_files, val_masks, test_files = ReadDirectories()
    train_files, train_masks = train_files[0:1200], train_masks[0:1200]
    test_files, test_masks = train_files[1200:], train_masks[1200:]

    # Get transformations from Transforms()
    transform_train, transform_test = Transforms()
    
    # Generate manual seed for reproducability
    torch.manual_seed(42)

    # Load dataset
    print("Loading datasets...")
    train_ds = DFUDataset(train_files, train_masks, transform=transform_train)
    val_ds = DFUDataset(val_files, val_masks, transform=transform_test)
    print("Complete.")

    # Initiate DataLoader
    batch_size, max_epochs, early_stopping, model, optimizer, scheduler, loss_fn, DeepLab = Configurations()
    train_iter = DataLoader(train_ds, batch_size, shuffle=True, pin_memory=True)
    val_iter = DataLoader(val_ds, batch_size, shuffle=True, pin_memory=True)
    print("Dataloader Initiated.")

    # Set device for running model
    if torch.cuda.is_available():
        device = 'cuda'
    elif torch.backends.mps.is_available():
        device = 'mps'
    else:
        device = 'cpu'
    print("Running model on device {}".format(device))

    # Initiate segmentation model for training
    model = model.to(device)

    # Model Training
    Train(model, max_epochs, optimizer, scheduler, loss_fn, train_iter, val_iter, device, early_stopping, DeepLab)


if __name__ == '__main__':
    main()

Reading files from directories.
Complete.
Loading datasets...
Complete.
Dataloader Initiated.
Running model on device cuda
Initial learning rate set to  0.01


100%|██████████| 25/25 [00:09<00:00,  2.65it/s]


Saved best model!
Epoch 0, Train Loss 0.930501, Train IOU 0.067282, Val Loss 0.914670, Val IOU 0.212370


100%|██████████| 25/25 [00:08<00:00,  2.91it/s]


Saved best model!
Epoch 1, Train Loss 0.845671, Train IOU 0.217307, Val Loss 0.815703, Val IOU 0.374449


...

100%|██████████| 25/25 [00:10<00:00,  2.31it/s]


Epoch 5, Train Loss 0.320999, Train IOU 0.563143, Val Loss 0.448714, Val IOU 0.497544


100%|██████████| 25/25 [00:09<00:00,  2.70it/s]


Saved best model!
Epoch 6, Train Loss 0.300410, Train IOU 0.578526, Val Loss 0.404608, Val IOU 0.533699


  4%|▍         | 4/100 [00:03<01:22,  1.17it/s]